### To do - August 25th
* Drop or fill NaN values: Non_motor assessments to do: solve issue of fillna not working...
* Add non 'TS features' as described below
* fill NaN values in the final merged df
* Do some mods on the time variables (INFODT,visits...)
* choose an input timesteps and do some padding/cropping/dropping
* Save final df as csv files in output/pre-processing

### Data Frame Merging Roadmap
#### 1. PD Feat
* DIAGFEAT
* EPWORTH
* LNSPD
* REMSLEEP
* SCOPAAUT
* COGATG
* HVLT
* MOCA
* GDSSHORT
* SDM
* STAI
* BENTONOD
* QUIPCS

#### 2. Gen Medical info
* GENPHYEX
* PENEURO
* PENEURO2
* VITAL
#### 3. PD Progression measurements (motors)
* NUPDRS1
* NUPDRS2P
* NUPDR3
* NUPDRS4
* (PASELT)
* MODSEADL

#### 4. Medications data
* PDMEDUSE
* (CURRCOND)
* SURGPD

#### 5. Non TS Features
* SCREEN - Gen Med info
* PDFEAT - PD feat
* MHXGEN - Gen Med info
* Biospecimen_analysis: ? 
* LUMBAR - Gen Med info
* PRIMDX - PD Feat
* PRODIAG - PD Feat
* to add: time since the last hospital visit - PD Feat

#### 6. Targets (from num3)
    *MSEADLG
    *NHY
    
#### Around 400 features!!

In [1]:
#imports 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import urllib.parse
from glob import glob
import os
import ntpath 
import zipfile

PPMI_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/'

In [2]:
def reading_csv_files(folder_path):
    # saving all the csv files in a dataframe
    import os
    import glob
    # glob.glob('*.csv') #find all the csv files in a pathname. 
    os.chdir(folder_path)
    csv_files = [i for i in glob.glob('*.csv')]

    # Reading each csv file and storing them in a dictionnary containing the file name and the dataframe
    dict_files={}
    files_names=[]
    for files in csv_files:
        df=pd.read_csv(files,engine='python')
        basepath, filename=ntpath.split(files)
        files_names.append(filename)
        dict_files[filename]=df

    # Calculating the # of rows and features for each dataframe and storing them in a list of tuples
    shape_df=[]
    for i in range(0,len(dict_files.keys())):
        nrows, ncols=(len(dict_files[files_names[i]]),len(dict_files[files_names[i]].columns))
        shape_df.append((nrows,ncols))

    return (shape_df,files_names,dict_files)

In [3]:
def number_of_patients(df):
    '''return the number of patients in each csv files'''
    if 'PATNO' in df.columns:
        patients=len(list(set(df['PATNO'])))
    else: 
        patients=0
    return patients

In [4]:
def number_of_events(df): 
    '''return the number of events covered by each csv file'''
    if 'EVENT_ID' in df.columns:
        events=len(list(set(df['EVENT_ID'])))
    else: 
        events=0
    return events

In [5]:
def features_selection(list_df_SEL):
    '''do features selection for each df,sel in list_df_sel (dictionnary)'''
    return 

In [6]:
def patients_selection(df, threshold=6):
    '''return a dataframe containing only the patients with number of visits > threshold'''
    if 'PATNO' in df.columns: 
        visits_number_by_pat=df.groupby('PATNO').size().sort_values(ascending=False)
        mask_sel=visits_number_by_pat>=threshold
        patients_sel=list(mask_sel[mask_sel.values==True].index)
        df=df.loc[df['PATNO'].isin(patients_sel),:]
        df=df.sort_values('PATNO')
        return (patients_sel, df)
    else:
        return 0

In [7]:
def PD_patients_selection(df):
    folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/1-Subject-Characteristics/'
    files_names=reading_csv_files(folder_path)[1]
    dict_files=reading_csv_files(folder_path)[2]

    ### Patient_Status - actually useful for selecting only the PD patients in each table
    PRODROMA=dict_files[files_names[0]]
    PD_cat=['PD','REGPD','GENPD']
    PD_patients=list(PRODROMA.loc[PRODROMA['RECRUITMENT_CAT'].isin(PD_cat),'PATNO'])
    sub_df=df.loc[df['PATNO'].isin(PD_patients),:]
    return (PD_patients,sub_df)

In [8]:
def table_analysis(list_df,df_names,threshold=6): 
    '''return a dataframe with: 
    -as rows: list of df
    -as columns: 
        - total number of observations=rows
        - total number of features=columns after features_selection - features_selection function 
        - number of events
        - number of PD patients by using PRODROMA info - number_of_patients function
        - number of patients with # of visits > threshold - number_of_patients function
        - shape of final df after selections of rows and columns
        -number of NaN values in the final df
        NB: takes as input the df with features selection. 
    '''   
    # initialisation df
    col=['observations_tot','features','events_num','patients_num','patients_sel','len_df_sel','NaN_values']
    df_df=pd.DataFrame(columns=col,index=df_names)

    # remplissage of dataframe for each row
    for i,df in enumerate(list_df): 
        df_df.iloc[i,:]['observations_tot']=len(df)
        df_df.iloc[i,:]['features']=len(df.columns)
        df_df.iloc[i,:]['events_num']=number_of_events(df)
        df_df.iloc[i,:]['patients_num']=number_of_patients(df)
        
        if not patients_selection(df)==0:
            df=patients_selection(df,threshold=threshold)[1]
            df=PD_patients_selection(df)[1]          
            df_df.iloc[i,:]['patients_sel']=len(set(df['PATNO']))
        else: 
            if 'PATNO' in df.columns:
                df=PD_patients_selection(df)[1]
                df_df.iloc[i,:]['patients_sel']=len(set(df['PATNO']))
            else:
                df_df.iloc[i,:]['patients_sel']=0            
        df_df.iloc[i,:]['len_df_sel']=len(df)
        df_df.iloc[i,:]['NaN_values']=(df.isnull().sum().sum())/(df_df.iloc[i,:]['len_df_sel']*df_df.iloc[i,:]['features'])
        
    return df_df

In [9]:
def INFODT_date(df):
    if 'INFODT' in df.columns: 
        df['INFODT_date']=df['INFODT'].apply(lambda x: datetime.strptime(x,'%m/%Y'))
    return df

In [10]:
def merge(df_left,df_right,on=['PATNO','EVENT_ID','INFODT'],how='inner'): 
    df=pd.merge(df_left,df_right,on=on,how=how)
    return df

In [11]:
def merge_multiple_df(list_df,on=['PATNO','EVENT_ID','INFODT'],how='inner'): 
    ## Dropping useless columns: 
    for df in list_df: 
        if all(a in ['REC_ID','PAG_NAME'] for a in list(df.columns)):
            df.drop(columns=['REC_ID','PAG_NAME'],inplace=True)
        elif 'REC_ID' in list(df.columns):
            df.drop(columns=['REC_ID'],inplace=True)
        elif 'PAG_NAME' in list(df.columns):
            df.drop(columns=['PAG_NAME'],inplace=True)

    ## Merging the PD features
    merged_df=merge(list_df[0],list_df[1],on=on,how=how)
    print((len(merged_df)),len(set(merged_df['PATNO'])))
    for df in list_df[2:]: 
        merged_df=merge(merged_df,df,on=on,how=how)
        print((len(merged_df)),len(set(merged_df['PATNO'])))
    return merged_df

### Loading and selecting features in tables

In [54]:
## 5-Medical History
### PDFEAT is important - keep it.

folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/5-Medical-History/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### 'Diagnostic_Features.csv': useful: Feat-PD - 1670 patients.
DIAGFEAT=dict_files[files_names[1]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID','INFODT',
       'DFSTROKE', 'DFRSKFCT', 'DFPRESNT', 'DFRPROG', 'DFSTATIC', 'DFHEMPRK',
       'DFAGESX', 'DFOTHCRS', 'DFRTREMP', 'DFRTREMA', 'DFPATREM',
       'DFOTHTRM', 'DFRIGIDP', 'DFRIGIDA', 'DFAXRIG', 'DFUNIRIG',
       'DFTONE', 'DFOTHRIG', 'DFBRADYP', 'DFBRADYA', 'DFAKINES',
       'DFBRPLUS', 'DFOTHABR', 'DFPGDIST', 'DFGAIT', 'DFFREEZ',
       'DFFALLS', 'DFOTHPG', 'DFPSYCH', 'DFCOGNIT', 'DFDYSTON',
       'DFCHOREA', 'DFMYOCLO', 'DFOTHHYP', 'DFHEMTRO', 'DFPSHYPO',
       'DFSEXDYS', 'DFURDYS', 'DFBWLDYS', 'DFOCULO', 'DFEYELID', 'DFNEURAB',
       'DFDOPRSP', 'DFRAPSPE', 'DFBULBAR', 'DFCTSCAN', 'DFMRI', 'DFATYP']
DIAGFEAT_sel=DIAGFEAT[SEL]


### General_Physical_Exam: useful: Feat-Gen - 2047 patients
GENPHYEX=dict_files[files_names[5]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID','INFODT',
       'PESEQ', 'ABNORM']
GENPHYEX_sel=GENPHYEX[SEL]

### 'General_Neurological_Exam.csv': useful: Feat-Gen - 2048 patients
PENEURO=dict_files[files_names[7]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID','INFODT',
       'MSRARSP','MSLARSP','MSRLRSP',
       'MSLLRSP', 'COFNRRSP', 'COFNLRSP',
       'COHSRRSP', 'COHSLRSP', 'SENRARSP',
       'SENLARSP', 'SENRLRSP', 'SENLLRSP',
       'RFLRARSP', 'RFLLARSP', 'RFLRLRSP',
       'RFLLLRSP', 'PLRRRSP', 'PLRLRSP']
PENEURO_sel=PENEURO[SEL]

### 'Neurological_Exam_-_Cranial_Nerves.csv': useful: Feat-PD - 2049 patients
PENEURO2=dict_files[files_names[13]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID','INFODT',
       'CN1RSP','CN2RSP', 'CN346RSP','CN5RSP',
     'CN7RSP', 'CN8RSP', 'CN910RSP',
       'CN11RSP', 'CN12RSP']
PENEURO2_sel=PENEURO2[SEL]


### 'Vital_Signs.csv': useful: Feat-Gen
### 2047 patients
VITAL=dict_files[files_names[15]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID','INFODT', 'SYSSUP', 'DIASUP', 'HRSUP',
       'SYSSTND', 'DIASTND', 'HRSTND']
VITAL_sel=VITAL[SEL]

## 6-Motor-Assessments - all of them can be useful. 
#* UPDRS1 assesses non-motor experiences of daily-living 
#* UPDRS2P assesses motor experiences of daily living
#* UPDRS3 assesses motor signs of PD. Includes Hoehn & Yahr Stage. 
#* UPDRS4 assesses motor complications, dyskinesias, & moto fluctuations using historical & objective information.

folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/6-Motor-Assessments/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### 'MDS_UPDRS_Part_I__Patient_Questionnaire.csv' - 2011 patients
NUPDR1SP=dict_files[files_names[0]]

### 'MDS_UPDRS_Part_I.csv' - 2010 patients - 
NUPDRS1=dict_files[files_names[2]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID','INFODT',
       'NP1COG', 'NP1HALL', 'NP1DPRS', 'NP1ANXS', 'NP1APAT',
       'NP1DDS']
NUPDRS1_sel=NUPDRS1[SEL]

### 'MDS_UPDRS_Part_II__Patient_Questionnaire.csv' - 2011 patients
NUPDRS2P=dict_files[files_names[11]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID','INFODT',
       'NP2SPCH', 'NP2SALV', 'NP2SWAL', 'NP2EAT', 'NP2DRES',
       'NP2HYGN', 'NP2HWRT', 'NP2HOBB', 'NP2TURN', 'NP2TRMR', 'NP2RISE',
       'NP2WALK', 'NP2FREZ']
NUPDRS2P_sel=NUPDRS2P[SEL]

### 'MDS_UPDRS_Part_III.csv' - 2010 patients
NUPDRS3=dict_files[files_names[7]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID','INFODT','NP3SPCH', 'NP3FACXP', 'NP3RIGN', 'NP3RIGRU',
       'NP3RIGLU', 'PN3RIGRL', 'NP3RIGLL', 'NP3FTAPR', 'NP3FTAPL', 'NP3HMOVR',
       'NP3HMOVL', 'NP3PRSPR', 'NP3PRSPL', 'NP3TTAPR', 'NP3TTAPL', 'NP3LGAGR',
       'NP3LGAGL', 'NP3RISNG', 'NP3GAIT', 'NP3FRZGT', 'NP3PSTBL', 'NP3POSTR',
       'NP3BRADY', 'NP3PTRMR', 'NP3PTRML', 'NP3KTRMR', 'NP3KTRML', 'NP3RTARU',
       'NP3RTALU', 'NP3RTARL', 'NP3RTALL', 'NP3RTALJ', 'NP3RTCON', 'DYSKPRES','NHY','ON_OFF_DOSE',
       'PD_MED_USE']
NUPDRS3_sel=NUPDRS3[SEL]

target_NHY=NUPDRS3[['PATNO','EVENT_ID','INFODT','NHY']]


### 'MDS_UPDRS_Part_IV.csv' - 860 patients
NUPDRS4=dict_files[files_names[5]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID','INFODT',
       'NP4WDYSK', 'NP4DYSKI', 'NP4OFF', 'NP4FLCTI', 'NP4FLCTX', 'NP4DYSTN']
NUPDRS4_sel=NUPDRS4[SEL]

### 'PASE_-_Household_Activity.csv' - 1248 patients
PASE=dict_files[files_names[3]]

### 'PASE_-_Leisure_Time_Activity.csv' - 1248 patients
PASELT=dict_files[files_names[10]]

### 'Modified_Schwab_+_England_ADL.csv' - 1837 patients
MODSEADL=dict_files[files_names[6]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID','INFODT',
       'MSEADLG']
MODSEADL=MODSEADL[SEL]
target_MSEADLG=MODSEADL[['PATNO','EVENT_ID','INFODT','MSEADLG']]

## 7-Non-motor-assessments - all of them can be useful but we might need to choose. 
### Do a correlation analysis and ask doctors which ones are the more relevant. 

folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/7-Non-motor-assessments/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### 'University_of_Pennsylvania_Smell_ID_Test.csv' - 1799 patients - OLFACTION
UPSIT=dict_files[files_names[0]]

### 'Epworth_Sleepiness_Scale.csv - 1343 patients - SLEEP. 
EPWORTH=dict_files[files_names[2]]

### 'Letter_-_Number_Sequencing__PD_.csv' - 1341 patients. - MEMORY. 
LNSPD=dict_files[files_names[3]]

### 'Olfactory_UPSIT.csv' - 4468 patients - OLFACTION. 
#### NB: only take the 2 last columns as features (TOTAL_CORRECT, UPSIIT_PRCNTGE)
OLFACT=dict_files[files_names[4]]
col_old=list(OLFACT.columns)
col_new=['PAT_NO', 'PROTOCOL_ID', 'INFODT', 'SCENT_10_RESPONSE',
       'SCENT_09_RESPONSE', 'SCENT_08_RESPONSE', 'SCENT_07_RESPONSE',
       'SCENT_06_RESPONSE', 'SCENT_05_RESPONSE', 'SCENT_04_RESPONSE',
       'SCENT_03_RESPONSE', 'SCENT_02_RESPONSE', 'SCENT_01_RESPONSE',
       'SCENT_20_RESPONSE', 'SCENT_19_RESPONSE', 'SCENT_18_RESPONSE',
       'SCENT_17_RESPONSE', 'SCENT_16_RESPONSE', 'SCENT_15_RESPONSE',
       'SCENT_14_RESPONSE', 'SCENT_13_RESPONSE', 'SCENT_12_RESPONSE',
       'SCENT_11_RESPONSE', 'SCENT_30_RESPONSE', 'SCENT_29_RESPONSE',
       'SCENT_28_RESPONSE', 'SCENT_27_RESPONSE', 'SCENT_26_RESPONSE',
       'SCENT_25_RESPONSE', 'SCENT_24_RESPONSE', 'SCENT_23_RESPONSE',
       'SCENT_22_RESPONSE', 'SCENT_21_RESPONSE', 'SCENT_40_RESPONSE',
       'SCENT_39_RESPONSE', 'SCENT_38_RESPONSE', 'SCENT_37_RESPONSE',
       'SCENT_36_RESPONSE', 'SCENT_35_RESPONSE', 'SCENT_34_RESPONSE',
       'SCENT_33_RESPONSE', 'SCENT_32_RESPONSE', 'SCENT_31_RESPONSE',
       'SCENT_10_CORRECT', 'SCENT_09_CORRECT', 'SCENT_08_CORRECT',
       'SCENT_07_CORRECT', 'SCENT_06_CORRECT', 'SCENT_05_CORRECT',
       'SCENT_04_CORRECT', 'SCENT_03_CORRECT', 'SCENT_02_CORRECT',
       'SCENT_01_CORRECT', 'SCENT_20_CORRECT', 'SCENT_19_CORRECT',
       'SCENT_18_CORRECT', 'SCENT_17_CORRECT', 'SCENT_16_CORRECT',
       'SCENT_15_CORRECT', 'SCENT_14_CORRECT', 'SCENT_13_CORRECT',
       'SCENT_12_CORRECT', 'SCENT_11_CORRECT', 'SCENT_30_CORRECT',
       'SCENT_29_CORRECT', 'SCENT_28_CORRECT', 'SCENT_27_CORRECT',
       'SCENT_26_CORRECT', 'SCENT_25_CORRECT', 'SCENT_24_CORRECT',
       'SCENT_23_CORRECT', 'SCENT_22_CORRECT', 'SCENT_21_CORRECT',
       'SCENT_40_CORRECT', 'SCENT_39_CORRECT', 'SCENT_38_CORRECT',
       'SCENT_37_CORRECT', 'SCENT_36_CORRECT', 'SCENT_35_CORRECT',
       'SCENT_34_CORRECT', 'SCENT_33_CORRECT', 'SCENT_32_CORRECT',
       'SCENT_31_CORRECT', 'TOTAL_CORRECT', 'UPSIIT_PRCNTGE']
col=dict(zip(col_old,col_new))
OLFACT.rename(columns=col,inplace=True)
SEL=['PAT_NO', 'PROTOCOL_ID', 'INFODT','TOTAL_CORRECT', 'UPSIIT_PRCNTGE']
OLFACT_sel=OLFACT[SEL]

### 'REM_Sleep_Disorder_Questionnaire.csv' - 1810 patients - SLEEP. 
REMSLEEP=dict_files[files_names[5]]

### 'Cognitive_Assessments.csv' - 1172 patients
COGTIME=dict_files[files_names[6]]

### 'SCOPA-AUT.csv' - 1345 patients - AUTOMATIC DYSFUNCTIONS
SCOPAAUT=dict_files[files_names[7]]

### Cognitive_Categorization.csv' - 1302 patients - DEMENTIA/MILD COGNITIVE IMPAIRMENT
COGCATG=dict_files[files_names[8]]

### 'Hopkins_Verbal_Learning_Test.csv' - 1341 patients - MEMORY
HVLT=dict_files[files_names[9]]

### 'Montreal_Cognitive_Assessment__MoCA_.csv' - 2054 patients - MILD COGNITIVE IMPAIRMENT - PD 
MOCA=dict_files[files_names[10]]
                
### 'Semantic_Fluency.csv' - 1342 patients - Lexical/Semantic Fluency
SFT=dict_files[files_names[11]]

### 'Geriatric_Depression_Scale__Short_.csv' - 1478 patients - DEPRESSION
GDSSHORT=dict_files[files_names[12]]

### 'Symbol_Digit_Modalities.csv' - 1340 patients - COGNITIVE IMPAIRMENT. 
SDM=dict_files[files_names[13]]

### 'State-Trait_Anxiety_Inventory.csv' - 1474 patients - DEPRESSION
STAI=dict_files[files_names[14]]

### 'Benton_Judgment_of_Line_Orientation.csv' - 1341 patients - SPATIAL PERCEPTION/ORIENTATION 
BENTONOD=dict_files[files_names[15]]

### 'QUIP_Current_Short.csv' - 1346 patients - COMPULSIVE DISORDERS
QUIPCS=dict_files[files_names[16]]

SEL_EP=['PATNO', 'EVENT_ID', 'INFODT', 'PTCGBOTH', 'ESS1', 'ESS2',
       'ESS3', 'ESS4', 'ESS5', 'ESS6', 'ESS7', 'ESS8']
SEL_LNSPD=['PATNO', 'EVENT_ID', 'INFODT','LNS_TOTRAW', 'AGE_ASSESS_LNS', 'DVS_LNS']
SEL_REM=['PATNO', 'EVENT_ID', 'INFODT', 'PTCGBOTH', 'DRMVIVID',
       'DRMAGRAC', 'DRMNOCTB', 'SLPLMBMV', 'SLPINJUR', 'DRMVERBL', 'DRMFIGHT',
       'DRMUMV', 'DRMOBJFL', 'MVAWAKEN', 'DRMREMEM', 'SLPDSTRB', 'STROKE',
       'HETRA', 'PARKISM', 'RLS', 'NARCLPSY', 'DEPRS', 'EPILEPSY', 'BRNINFM']
SEL_SCOPA=['PATNO', 'EVENT_ID', 'INFODT', 'PTCGBOTH', 'SCAU1', 'SCAU2',
       'SCAU3', 'SCAU4', 'SCAU5', 'SCAU6', 'SCAU7', 'SCAU8', 'SCAU9', 'SCAU10',
       'SCAU11', 'SCAU12', 'SCAU13', 'SCAU14', 'SCAU15', 'SCAU16', 'SCAU17',
       'SCAU18', 'SCAU19', 'SCAU20', 'SCAU21','SCAU26B','SCAU26C','SCAU26D']
SEL_COG=['PATNO', 'EVENT_ID', 'INFODT', 'PTCGBOTH', 'COGDECLN',
       'FNCDTCOG', 'COGSTATE', 'COGDXCL', 'RVWNPSY']
SEL_HVLT=['PATNO', 'EVENT_ID', 'INFODT', 'HVLTRT1', 'HVLTRT2',
       'HVLTRT3', 'HVLTRDLY', 'HVLTREC', 'HVLTFPRL', 'HVLTFPUN', 'HVLTVRSN','DVT_TOTAL_RECALL',
        'DVT_DELAYED_RECALL','DVT_RETENTION', 'DVT_RECOG_DISC_INDEX']
SEL_MOCA=['PATNO', 'EVENT_ID', 'INFODT', 'MCAALTTM', 'MCACUBE',
       'MCACLCKC', 'MCACLCKN', 'MCACLCKH', 'MCALION', 'MCARHINO', 'MCACAMEL',
       'MCAFDS', 'MCABDS', 'MCAVIGIL', 'MCASER7', 'MCASNTNC', 'MCAVFNUM',
       'MCAVF', 'MCAABSTR', 'MCAREC1', 'MCAREC2', 'MCAREC3', 'MCAREC4',
       'MCAREC5','MCATOT']
SEL_GDS=['PATNO', 'EVENT_ID', 'INFODT', 'GDSSATIS', 'GDSDROPD',
       'GDSEMPTY', 'GDSBORED', 'GDSGSPIR', 'GDSAFRAD', 'GDSHAPPY', 'GDSHLPLS',
       'GDSHOME', 'GDSMEMRY', 'GDSALIVE', 'GDSWRTLS', 'GDSENRGY', 'GDSHOPLS',
       'GDSBETER']
SEL_SDM=['PATNO', 'EVENT_ID', 'INFODT', 'SDMTOTAL', 'SDMTVRSN','DVSD_SDM', 'DVT_SDM']
SEL_STAI=['PATNO', 'EVENT_ID', 'INFODT', 'STAIAD1', 'STAIAD2',
       'STAIAD3', 'STAIAD4', 'STAIAD5', 'STAIAD6', 'STAIAD7', 'STAIAD8',
       'STAIAD9', 'STAIAD10', 'STAIAD11', 'STAIAD12', 'STAIAD13', 'STAIAD14',
       'STAIAD15', 'STAIAD16', 'STAIAD17', 'STAIAD18', 'STAIAD19', 'STAIAD20',
       'STAIAD21', 'STAIAD22', 'STAIAD23', 'STAIAD24', 'STAIAD25', 'STAIAD26',
       'STAIAD27', 'STAIAD28', 'STAIAD29', 'STAIAD30', 'STAIAD31', 'STAIAD32',
       'STAIAD33', 'STAIAD34', 'STAIAD35', 'STAIAD36', 'STAIAD37', 'STAIAD38',
       'STAIAD39', 'STAIAD40']
SEL_BENTON=['PATNO', 'EVENT_ID', 'INFODT','JLO_TOTRAW', 'JLO_TOTCALC',
       'AGE_ASSESS_JLO', 'DVS_JLO_MSSA', 'DVS_JLO_MSSAE']
SEL_QUIPCS=['PATNO', 'EVENT_ID', 'INFODT', 'PTINBOTH', 'TMGAMBLE',
       'CNTRLGMB', 'TMSEX', 'CNTRLSEX', 'TMBUY', 'CNTRLBUY', 'TMEAT',
       'CNTRLEAT', 'TMTORACT', 'TMTMTACT', 'TMTRWD', 'TMDISMED', 'CNTRLDSM']

NM_assessments_df=[EPWORTH,LNSPD,REMSLEEP,SCOPAAUT,COGCATG,HVLT,MOCA,GDSSHORT,SDM,STAI,BENTONOD,QUIPCS]
SEL_NM=[SEL_EP,SEL_LNSPD,SEL_REM,SEL_SCOPA,SEL_COG,SEL_HVLT,SEL_MOCA,SEL_GDS,
        SEL_SDM,SEL_STAI,SEL_BENTON,SEL_QUIPCS]
NM_sel_df=list()
for sel,df in zip(SEL_NM,NM_assessments_df):
        NM_sel_df.append(df[sel])
        
[EPWORTH,LNSPD,REMSLEEP,SCOPAAUT,COGCATG,HVLT,MOCA,GDSSHORT,SDM,STAI,BENTONOD,QUIPCS]=[i for i in NM_sel_df]

folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/5-Medical-History/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### Use of PD Medication: useful: Meds - but not to be used for now.
PDMEDUSE=dict_files[files_names[4]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'INFODT',
       'PDMEDYN', 'ONLDOPA', 'ONDOPAG', 'ONOTHER', 'FULNUPDR', 'PDMEDDT']
PDMEDUSE_sel=PDMEDUSE[SEL]

### 'Concomitant_Medications.csv': useful: Meds - no time to exploit this stuff. 
CMED=dict_files[files_names[6]]

### 'Current_Medical_Conditions_Log.csv': Meds
CURRCOND=dict_files[files_names[8]]
CURRCOND.columns
SEL=['REC_ID', 'PATNO', 'EVENT_ID','SEQNO',
       'CONDCAT', 'DIAGYR','CONDTERM', 'RESOLVD', 'RESYR',
       'HLGTNAME', 'HLT_NAME','PT_CODE','SOCABBR1', 'VMEDDRA']
CURRCOND_sel=CURRCOND[SEL]

### 'Surgery_for_Parkinson_Disease.csv': useful: Meds - but not to be used for now.
SURGPD=dict_files[files_names[12]]
SURGPD.columns
SEL=['REC_ID','PATNO', 'EVENT_ID','INFODT',
       'PDSURG', 'PDSURGDT', 'PDSURGTP']
SURGPD_sel=SURGPD[SEL]

## Non TS features. 
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/1-Subject-Characteristics/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### Screening__Demographics - 2144 patients (all pop)
#### NOT TS DATA
SCREEN=dict_files[files_names[3]]
SEL=['REC_ID', 'PATNO','BIRTHDT', 'GENDER','P3GRP','CURRENT_APPRDX','APPRDX']
SCREEN_sel=SCREEN[SEL]
zeros=pd.Series(np.zeros(len(SCREEN_sel)))
SCREEN_sel['P3GRP'].fillna(value=zeros,inplace=True)
# fill NaN values of P3GRP with '0' (non podroma people)
#SCREEN_sel.isnull().sum()

folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/5-Medical-History/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### 'PD_Features.csv': useful: Feat-PD - caution: these are actually not timeseries but examination only done at screening (SC), BL & V04
### 1017 patients
PDFEAT=dict_files[files_names[14]]
SEL=['REC_ID', 'PATNO', 'SXMO',
       'SXYEAR', 'PDDXDT', 'DXTREMOR', 'DXRIGID', 'DXBRADY',
       'DXPOSINS', 'DXOTHSX','DOMSIDE']
PDFEAT_sel=PDFEAT[SEL]

### 'General_Medical_History.csv': useful: Feat-Gen - 2067 patients
#### NOT TS DATA...
MHXGEN=dict_files[files_names[3]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID','INFODT','MHROW']
# I need probably to select less features here
MHXGEN_sel=MHXGEN[SEL]

## Biospecimen
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/2-Biospecimen/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### Pilot_Biospecimen_Analysis_Results - useful: Feat-bio - 664 patients...
Biospecimen_Analysis=dict_files[files_names[6]]
SEL=['PATNO', 'DIAGNOSIS', 'CLINICAL_EVENT','TESTNAME',
       'TESTVALUE']
Biospecimen_Analysis_sel=Biospecimen_Analysis[SEL]
Biospecimen_Analysis_sel.rename(columns={'CLINICAL_EVENT':'EVENT_ID'},inplace=True)

### Lumbar_Puncture_Sample_Collection - useful: Feat-bio - 1344 patients
LUMBAR=dict_files[files_names[9]]

SEL=['REC_ID','PATNO', 'EVENT_ID','INFODT','CSFSPNRT',
     'SMPDSCRD','RBCRSLT','WBCRSLT','TOPRRSLT','TGLCRSLT']
LUMBAR_sel=LUMBAR[SEL]

### Additionnal feature: PRIMDX. 
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/3-Enrollment/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### 'Primary_Diagnosis.csv' - useful: Feat-Gen - 789 patients... (only PD patients?)
PRIMDXPD=dict_files[files_names[8]]
SEL=['REC_ID','PATNO', 'EVENT_ID','INFODT',
       'PRIMDIAG']
print(number_of_patients(PRIMDXPD))
PRIMDXPD_sel=PRIMDXPD[SEL]

### 'Prodromal_Diagnostic_Questionnaire.csv': useful: Feat-Gen (PRIMDIAG & PSLVL2) 
#### NB: PSLV2 evaluates the proba of the patient being PD: might be useful for early PD. But not to be used in the main analysis as a feature (only for Prodromal & Genetic Cohort - i.e 800 subjects) 
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/5-Medical-History/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

PRODIAG=dict_files[files_names[0]]
SEL=['REC_ID', 'F_STATUS', 'PATNO', 'EVENT_ID','PRIMDIAG', 'PSLVL2']
PRODIAG_sel=PRODIAG[SEL]

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5430: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



789


In [13]:
SCREEN_sel.head()
zeros=pd.Series(np.zeros(len(SCREEN_sel)))
SCREEN_sel['P3GRP'].fillna(value=zeros,inplace=True)
# fill NaN values of P3GRP with '0' (non podroma people)
#SCREEN_sel.isnull().sum()

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [14]:
SCREEN_sel.head()

,REC_ID,PATNO,BIRTHDT,GENDER,P3GRP,CURRENT_APPRDX,APPRDX
0,224392801,3400,1971.0,0.0,0.0,1.0,1.0
1,224394301,3401,1954.0,1.0,0.0,2.0,2.0
2,224398401,3402,1964.0,2.0,0.0,3.0,3.0
3,224400201,3403,1941.0,2.0,0.0,1.0,1.0
4,224718101,3404,1954.0,0.0,0.0,2.0,2.0


In [15]:
LUMBAR_sel.head()
LUMBAR_sel.isnull().sum()
# Fill NaN values...
# Fill it with the average for humans (Blood cell count, glucose, protein rate, spin rate)
# Bloody tap is a binary variable, so fill it with 0 or 1.

REC_ID         0
PATNO          0
EVENT_ID       0
INFODT         0
CSFSPNRT    1365
SMPDSCRD    1360
RBCRSLT     1717
WBCRSLT     1701
TOPRRSLT    1706
TGLCRSLT    1712
dtype: int64

In [16]:
PRIMDXPD_sel['PSLVL2']=0
PRIMDX_PRODIAG=pd.concat([PRIMDXPD_sel,PRODIAG_sel],sort=False)
#PRIMDX_PRODIAG.tail()
#len(set(PRIMDX_PRODIAG.PATNO))
# Fill NaN values for 'INFODT' by extrapolating from other tables.

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


## General Medical Records df

In [53]:
Medical_Gen_df=[GENPHYEX_sel,PENEURO_sel,PENEURO2_sel,VITAL_sel]
Medical_General=merge_multiple_df(Medical_Gen_df)
Medical_General=merge(Medical_General,LUMBAR_sel,how='left')
print(len(Medical_General),len(set(Medical_General.PATNO)))
Medical_General=merge(Medical_General,MHXGEN_sel,how='left')
print(len(Medical_General),len(set(Medical_General.PATNO)))

Medical_General.rename(columns={'EVENT_ID_x':'EVENT_ID'},inplace=True)
Medical_General.drop(columns=['REC_ID_x','REC_ID_y'],inplace=True)



Medical_General['ABNORM'].fillna(0,inplace=True)
Medical_General['MHROW'].fillna(0,inplace=True)
Medical_General['SMPDSCRD'].fillna(0,inplace=True)

# filling NaN values with col average:
col_averages=list()
colToAverage=['SYSSUP', 'DIASUP', 'HRSUP', 'SYSSTND', 'DIASTND', 'HRSTND',
       'CSFSPNRT','RBCRSLT', 'WBCRSLT', 'TOPRRSLT', 'TGLCRSLT']
for col in colToAverage:
    colAv=round(Medical_General[col].mean(),1)
    col_averages.append(colAv)
    Medical_General[col].fillna(colAv,inplace=True)
    
# dropping the rest of NaN values:
Medical_General.dropna(inplace=True)

Medical_General.head()

24349 2044
24349 2044
21632 1798
21632 1798
576835 1798


,PATNO,EVENT_ID,INFODT,PESEQ,ABNORM,MSRARSP,MSLARSP,MSRLRSP,MSLLRSP,COFNRRSP,...,SYSSTND,DIASTND,HRSTND,CSFSPNRT,SMPDSCRD,RBCRSLT,WBCRSLT,TOPRRSLT,TGLCRSLT,MHROW
0,3402,SC,06/2010,3,0.0,0,0,0,0.0,0,...,127.0,86.0,64.0,2000.0,0,18.7,1.0,47.2,62.3,1.0
1,3402,SC,06/2010,3,0.0,0,0,0,0.0,0,...,127.0,86.0,64.0,2000.0,0,18.7,1.0,47.2,62.3,1.0
2,3402,SC,06/2010,3,0.0,0,0,0,0.0,0,...,127.0,86.0,64.0,2000.0,0,18.7,1.0,47.2,62.3,2.0
3,3402,SC,06/2010,3,0.0,0,0,0,0.0,0,...,127.0,86.0,64.0,2000.0,0,18.7,1.0,47.2,62.3,1.0
4,3402,SC,06/2010,3,0.0,0,0,0,0.0,0,...,127.0,86.0,64.0,2000.0,0,18.7,1.0,47.2,62.3,1.0


## PD features

In [52]:
print(len(DIAGFEAT_sel),len(set(DIAGFEAT_sel.PATNO)))
PD_features_df=[DIAGFEAT_sel,PRIMDX_PRODIAG]
PD_features=merge_multiple_df(PD_features_df,how='left')
PD_features=merge(PD_features,Biospecimen_Analysis_sel,on=['PATNO','EVENT_ID'],how='left')
PD_features.drop(columns='F_STATUS',inplace=True)
PD_features.head()

# fill PRIMDIAG with '1' if PD, '0' otherwise
mask_na=[i for i in range(len(PD_features)) if PD_features['PRIMDIAG'].isnull()[i]==True]
PD_features_na=PD_features.iloc[mask_na,:]
PD_patients_na=PD_patients_selection(PD_features_na)[0]
mask_na_PD=PD_features_na['PATNO'].isin(PD_patients_na)
fillNa_PRIMDIAG=list(np.where(mask_na_PD,1,0))
ser_na_PD=pd.Series(data=fillNa_PRIMDIAG,index=mask_na)

PD_features['PRIMDIAG'].fillna(ser_na_PD,inplace=True)
PD_features['PSLVL2'].fillna(0,inplace=True)
PD_features['DIAGNOSIS'].fillna(0,inplace=True)
PD_features['TESTNAME'].fillna(0,inplace=True)
PD_features['TESTVALUE'].fillna(0,inplace=True)

PD_features.dropna(inplace=True)
#PD_features.isnull().sum()

PD_features.head()

5739 1670
5739 1670


,PATNO,EVENT_ID,INFODT,DFSTROKE,DFRSKFCT,DFPRESNT,DFRPROG,DFSTATIC,DFHEMPRK,DFAGESX,...,DFRAPSPE,DFBULBAR,DFCTSCAN,DFMRI,DFATYP,PRIMDIAG,PSLVL2,DIAGNOSIS,TESTNAME,TESTVALUE
0,3428,V04,08/2012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0.0,17.0,0.0,Control,CSF Hemoglobin,below detection limit
1,3433,V04,09/2012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0.0,1.0,0.0,0,0,0
2,3166,V04,10/2012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,N,0,0.0,1.0,0.0,0,0,0
3,3162,V04,10/2012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,N,N,0.0,1.0,0.0,0,0,0
4,3432,ST,08/2012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0.0,1.0,0.0,PD,CSF Hemoglobin,43.33


## PD Progression Measurements df

In [51]:
PD_Progression_df=[NUPDRS1_sel,NUPDRS2P_sel,NUPDRS3_sel]
# Add PASELT? : if so you need to add a 'INFODT'field...
PD_Progression_motor=merge_multiple_df(PD_Progression_df)
PD_Progression_motor=merge(PD_Progression_motor,NUPDRS4_sel,how='left')
PD_Progression_motor.drop(columns='NHY',inplace=True)
PD_Progression_motor['ON_OFF_DOSE'].fillna(value=0,inplace=True)
# Fill ON_OFF_DOSE with 0 when NaN.
# Change with how='left' for NH3.
PD_Progression_motor.head()

col=['NP4WDYSK', 'NP4DYSKI', 'NP4OFF',
       'NP4FLCTI', 'NP4FLCTX', 'NP4DYSTN']
for cols in col:
    PD_Progression_motor[cols].fillna(value=0,inplace=True)

for col in list(PD_Progression_motor.columns):
    PD_Progression_motor[col].fillna(0,inplace=True)

PD_Progression_motor.head()

11206 2009
12639 2007


,PATNO,EVENT_ID,INFODT,NP1COG,NP1HALL,NP1DPRS,NP1ANXS,NP1APAT,NP1DDS,NP2SPCH,...,DYSKPRES,ON_OFF_DOSE,PD_MED_USE,REC_ID,NP4WDYSK,NP4DYSKI,NP4OFF,NP4FLCTI,NP4FLCTX,NP4DYSTN
0,3402,SC,07/2010,0.0,0.0,1.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3400,BL,07/2010,0.0,0.0,2.0,3.0,2.0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3403,SC,07/2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3404,BL,07/2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3406,SC,07/2010,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Non-motor assessments df

In [74]:
NM_assessments=merge_multiple_df(NM_sel_df)
NM_assessments.head()
#TMDISMED and CNTRLDSM to remove or encode?

# filling NaN values with col average:
col_averages=list()
colToAverage=['DVT_SDM','DVSD_SDM','JLO_TOTRAW','JLO_TOTCALC']
for col in colToAverage:
    colAv=round(NM_assessments[col].mean(),1)
    col_averages.append(colAv)

dict_na=dict(zip(colToAverage,col_averages))
NM_assessments.fillna(value=dict_na,inplace=True)
NM_assessments.head()
NM_assessments.isnull().sum().sort_values(ascending=False)

5240 1333
5233 1333
5228 1333
4424 1293
4420 1293
3614 1091
3610 1091
3602 1089
3593 1089
3585 1089
3579 1088


DVT_SDM                 80
DVSD_SDM                80
SCAU26D                 16
JLO_TOTRAW              11
JLO_TOTCALC             11
AGE_ASSESS_JLO          11
DVS_JLO_MSSA            11
DVS_JLO_MSSAE           11
DVT_RECOG_DISC_INDEX    10
HVLTFPUN                 9
HVLTREC                  9
DRMNOCTB                 9
HVLTFPRL                 9
STAIAD24                 8
STAIAD25                 7
STAIAD36                 7
STAIAD19                 6
STAIAD28                 6
STAIAD23                 6
STAIAD20                 6
SCAU7                    6
STAIAD17                 6
STAIAD16                 6
SCAU14                   6
STAIAD30                 6
STAIAD14                 6
STAIAD8                  6
STAIAD6                  6
STAIAD29                 6
STAIAD18                 6
                        ..
GDSHAPPY                 1
MVAWAKEN                 1
SDMTVRSN                 1
DRMOBJFL                 1
PARKISM                  1
DRMREMEM                 1
S

## Medications data df - 0.43 % of NaN values!! > use fillna values

In [49]:
medications_df=[PDMEDUSE_sel,SURGPD_sel]
medications_data=merge_multiple_df(medications_df,how='left')
print(medications_data.isnull().sum().sum()/(len(medications_data)*len(medications_data.columns)))
medications_data.head()

medications_data.columns

ColNaN_0=['PDMEDYN', 'ONLDOPA', 'ONDOPAG','ONOTHER', 'FULNUPDR', 'PDMEDDT', 'PDSURG', 'PDSURGDT']
for col in ColNaN_0:
    medications_data[col].fillna(0,inplace=True)
medications_data['PDSURGTP'].fillna(4,inplace=True)
#medications_data.isnull().sum()
medications_data.head()

6668 909
0.4260272945410918


,PATNO,EVENT_ID,INFODT,PDMEDYN,ONLDOPA,ONDOPAG,ONOTHER,FULNUPDR,PDMEDDT,PDSURG,PDSURGDT,PDSURGTP
0,3403,V01,10/2010,0,0.0,0.0,0.0,1,0,0.0,0,4.0
1,3400,V01,10/2010,0,0.0,0.0,0.0,1,0,0.0,0,4.0
2,3406,V01,11/2010,0,0.0,0.0,0.0,1,0,0.0,0,4.0
3,3400,ST,12/2010,0,0.0,0.0,0.0,1,0,0.0,0,4.0
4,3407,V01,01/2011,0,0.0,0.0,0.0,1,0,0.0,0,4.0


## Targets df - MSEADLG & Hoehn & Yahr Scale

In [56]:
targets=pd.merge(target_NHY,target_MSEADLG,on=['PATNO','EVENT_ID','INFODT'])
targets.dropna(inplace=True)
targets.isnull().sum()
# NaN values probably to be dropped. 
targets.head()

,PATNO,EVENT_ID,INFODT,NHY,MSEADLG
0,3506,V03,10/2011,1.0,90.0
1,3419,V02,12/2011,1.0,95.0
2,3762,BL,11/2011,2.0,100.0
3,3552,V03,10/2011,2.0,90.0
4,3462,V01,12/2011,2.0,80.0


In [43]:
list_df=[PD_features,NM_assessments,Medical_General,PD_Progression_motor,medications_data,targets]
df_names=['PD_features','Non-Motor_assessments','Medical_General','PD_Progression_motor','medications_data','targets']
dataset_analysis_final_df6=table_analysis(list_df,df_names)
dataset_analysis_final_df4=table_analysis(list_df,df_names,4)
dataset_analysis_final_df2=table_analysis(list_df,df_names,2)

print(dataset_analysis_final_df2['features'].sum()-5*3-2+19)
dataset_analysis_final_df2

363


,observations_tot,features,events_num,patients_num,patients_sel,len_df_sel,NaN_values
PD_features,5722,56,19,1668,681,3091,0
Non-Motor_assessments,3579,179,12,1088,533,2175,0.00107365
Medical_General,576824,45,8,1797,943,303055,0
PD_Progression_motor,12639,65,20,2007,814,8811,0
medications_data,6668,12,21,909,697,5969,0
targets,12057,4,0,1831,814,9171,0


In [25]:
def sel_for_plot(df,rowSel, colSel,num):
    df=df.loc[rowSel,colSel]
    NewCols=[c+'{}'.format(num) for c in colSel]
    colRenamed=dict(zip(colSel,NewCols))
    df.rename(columns=colRenamed,inplace=True)
    return df

rowSel=df_names
colSel=['patients_sel','len_df_sel']
df_to_plot6=sel_for_plot(dataset_analysis_final_df6,rowSel=rowSel,colSel=colSel,num=6)
df_to_plot4=sel_for_plot(dataset_analysis_final_df4,rowSel=rowSel,colSel=colSel,num=4)
df_to_plot2=sel_for_plot(dataset_analysis_final_df2,rowSel=rowSel,colSel=colSel,num=2)

df_to_plot=pd.merge(df_to_plot2,df_to_plot4,left_index=True,right_index=True)
df_to_plot=pd.merge(df_to_plot,df_to_plot6,left_index=True,right_index=True)
df_to_plot=df_to_plot.T
df_to_plot['Measure']=list(np.tile(('len_patients','len_df_sel'),3))
df_to_plot['Visits_threshold']=list(np.repeat((2,4,6),2))

def select_measure(df,rowSel): 
    df=df.loc[rowSel,:]
    df.drop(columns='Measure',inplace=True)
    df.rename(index=dict(zip(list(df.index),list(df['Visits_threshold'].values))),inplace=True)
    df.drop(columns='Visits_threshold',inplace=True)
    return df

dfPlot_patients=select_measure(df_to_plot,['patients_sel2','patients_sel4','patients_sel6'])

dfPlot_samples=select_measure(df_to_plot,['len_df_sel2','len_df_sel4','len_df_sel6'])
dfPlot_samples

,PD_features,Non-Motor_assessments,Medical_General,PD_Progression_motor,medications_data,targets
2,3091,2175,303055,8811,5969,9171
4,2600,1692,303055,8473,5745,8879
6,1280,625,303055,8177,5324,8614


In [26]:
import cufflinks as cf
# Configure it to work offline:
cf.go_offline(connected=True)

dfPlot_patients.iplot(kind='bar',
                              bins=list(dfPlot_patients.index),
                              title='number of patients per table for each number of visits threshold',
                              xTitle='number of visits thresold',
                            yTitle='number of patients per table')

In [27]:
dfPlot_samples2=dfPlot_samples.drop(columns='Medical_General')
dfPlot_samples2.iplot(kind='bar',
                              bins=list(dfPlot_samples2.index),
                              title='number of samples per table for each number of visits threshold',
                              xTitle='number of visits thresold',
                            yTitle='number of samples per table')

## Merging all df

In [28]:
intersect=list(((set(PD_features['PATNO'])) & (set(Medical_General['PATNO']))))
len(intersect)

1420

In [63]:
## issue to solve!!! 
all_features=[Medical_General,targets,PD_Progression_motor,medications_data,PD_features,NM_assessments]
df_allFeatures=merge_multiple_df(all_features,how='left')
df_allFeatures.head()

576934 1797
577154 1797
577154 1797
577154 1797
577154 1797


,PATNO,EVENT_ID,INFODT,PESEQ,ABNORM,MSRARSP,MSLARSP,MSRLRSP,MSLLRSP,COFNRRSP,...,CNTRLSEX,TMBUY,CNTRLBUY,TMEAT,CNTRLEAT,TMTORACT,TMTMTACT,TMTRWD,TMDISMED,CNTRLDSM
0,3402,SC,06/2010,3,0.0,0,0,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3402,SC,06/2010,3,0.0,0,0,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3402,SC,06/2010,3,0.0,0,0,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3402,SC,06/2010,3,0.0,0,0,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3402,SC,06/2010,3,0.0,0,0,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
df_allFeatures.isnull().sum().sort_values(ascending=False).tail(100)

# to fill NaN values: 
#*targets: average over patients_type (PD...)
#*PD_progression_motor: idem. 
#*medications_data: mostly 0
#* PD_features: 0 and average
#*NM_assessments: mostly 0. 

NP3RIGLU    71842
NP3HMOVL    71842
NP3RIGN     71842
NP3FACXP    71842
NP3SPCH     71842
NP2FREZ     71842
NP2WALK     71842
NP2RISE     71842
NP2TURN     71842
NP1HALL     71842
NP2HOBB     71842
NP2HWRT     71842
NP2HYGN     71842
NP2DRES     71842
NP2EAT      71842
NP2SWAL     71842
NP2SALV     71842
NP2SPCH     71842
NP1DDS      71842
NP1APAT     71842
NP1ANXS     71842
NP1DPRS     71842
NP3HMOVR    71842
NP3RIGRU    71842
NP3PRSPR    71842
NP3RTARU    71842
NP3PRSPL    71842
NP4DYSTN    71842
NP4FLCTX    71842
NP4FLCTI    71842
            ...  
MSLARSP         0
ABNORM          0
PESEQ           0
INFODT          0
EVENT_ID        0
PLRLRSP         0
PLRRRSP         0
HRSTND          0
CN1RSP          0
CN2RSP          0
MHROW           0
TGLCRSLT        0
TOPRRSLT        0
WBCRSLT         0
RBCRSLT         0
SMPDSCRD        0
CSFSPNRT        0
DIASTND         0
SYSSTND         0
HRSUP           0
DIASUP          0
SYSSUP          0
CN12RSP         0
CN11RSP         0
CN910RSP  